In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns


from copy import deepcopy

import statsmodels.api as sm
from datetime import datetime

import random
random.seed(0)
pd.set_option('display.max_columns', 500)

### create ready to use data frame for flask

In [180]:
#the data frame is sorted by support
rules=pd.read_csv("vendor_cate_rules.csv").drop('Unnamed: 0',axis=1).sort_values('support',ascending=False)

In [407]:
#another data frame sorted by confidence
rulec=pd.read_csv("vendor_cate_rules.csv").drop('Unnamed: 0',axis=1).sort_values('confidence',ascending=False)

In [122]:
print(rules.shape)
rules.head()

(2138, 6)


,lhs,rhs,support,confidence,lift,count
4,{},{us jewelry house:necklace},0.507561,0.507561,1.000000,1846
3,{},{us jewelry house:earring},0.391531,0.391531,1.000000,1424
2,{},{us jewelry house:bracelet},0.275502,0.275502,1.000000,1002
433,{us jewelry house:earring},{us jewelry house:necklace},0.251306,0.641854,1.264584,914
434,{us jewelry house:necklace},{us jewelry house:earring},0.251306,0.495125,1.264584,914


In [408]:
print(rulec.shape)
rulec.head()

(2138, 6)


,lhs,rhs,support,confidence,lift,count
9,{viola:necklace},{us jewelry house:necklace},0.025296,0.968421,1.907989,92
473,"{hush:necklace,romance:necklace}",{us jewelry house:necklace},0.023921,0.956044,1.883603,87
1561,"{andante:necklace,us jewelry house:bracelet,us...",{us jewelry house:necklace},0.023096,0.954545,1.880651,84
1536,"{pomina:necklace,us jewelry house:bracelet,us ...",{us jewelry house:necklace},0.020071,0.948052,1.867857,73
5,{image:necklace},{us jewelry house:necklace},0.022271,0.941860,1.855659,81


In [409]:
rules['lhs'].iloc[0]

'{}'

In [410]:
#remove the empty rules
rules_=rules[rules['lhs']!='{}']
rulec_=rulec[rulec['lhs']!='{}']

In [411]:
#remove the {}
rules_['lhs']=rules_['lhs'].replace('[{}]', '', regex=True)
rules_['rhs']=rules_['rhs'].replace('[{}]', '', regex=True)
####
rulec_['lhs']=rulec_['lhs'].replace('[{}]', '', regex=True)
rulec_['rhs']=rulec_['rhs'].replace('[{}]', '', regex=True)

/Users/jialanzhu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/jialanzhu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/jialanzhu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

In [179]:
rules_.head()

,lhs,rhs,support,confidence,lift,count
433,us jewelry house:earring,us jewelry house:necklace,0.251306,0.641854,1.264584,914
434,us jewelry house:necklace,us jewelry house:earring,0.251306,0.495125,1.264584,914
431,us jewelry house:bracelet,us jewelry house:necklace,0.204014,0.740519,1.458975,742
432,us jewelry house:necklace,us jewelry house:bracelet,0.204014,0.401950,1.458975,742
429,us jewelry house:bracelet,us jewelry house:earring,0.162497,0.589820,1.506444,591


In [412]:
rulec_.head()

,lhs,rhs,support,confidence,lift,count
9,viola:necklace,us jewelry house:necklace,0.025296,0.968421,1.907989,92
473,"hush:necklace,romance:necklace",us jewelry house:necklace,0.023921,0.956044,1.883603,87
1561,"andante:necklace,us jewelry house:bracelet,us ...",us jewelry house:necklace,0.023096,0.954545,1.880651,84
1536,"pomina:necklace,us jewelry house:bracelet,us j...",us jewelry house:necklace,0.020071,0.948052,1.867857,73
5,image:necklace,us jewelry house:necklace,0.022271,0.941860,1.855659,81


### create all item dictionary vendor:category : items

In [136]:
usjh_all=pd.read_csv("USJH_rules.csv").drop(['Unnamed: 0','X','Unnamed..0','Unnamed..0.1'], axis=1)

/Users/jialanzhu/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [138]:
usjh_all.head()

,name,financial_status,paid_at,fulfillment_status,accepts_marketing,currency,subtotal,shipping,taxes,total,discount_code,discount_amount,shipping_method,created_at,lineitem_quantity,lineitem_name,lineitem_price,lineitem_compare_at_price,lineitem_sku,lineitem_fulfillment_status,shipping_street,shipping_address1,shipping_company,shipping_city,shipping_zip,shipping_province,shipping_country,notes,cancelled,payment_method,refunded_amount,vendor,risk_level,source,color,category,vendor_category
0,#4746,paid,2018-07-16 12:31:00,unfulfilled,yes,usd,100.5,13.64,0.0,114.14,nodiscount,0.0,fedex ground,2018-07-16 12:31:00,1,turquoise crystal rhinestone ring - gdtq,3.50,3.50,16646,pending,475 calle retama,475 calle retama,t-trendz,brownsville,78520,tx,us,nonotes,1,shopify payments,0.0,us jewelry house,low,web,gdtq,ring,us jewelry house:ring
1,#4746,paid,2018-07-16 12:31:00,unfulfilled,yes,usd,100.5,13.64,0.0,114.14,nodiscount,0.0,fedex ground,2018-07-16 12:31:00,1,turquoise crystal rhinestone ring - gdtq,3.25,3.25,16649,pending,475 calle retama,475 calle retama,t-trendz,brownsville,78520,tx,us,nonotes,1,shopify payments,0.0,us jewelry house,low,web,gdtq,ring,us jewelry house:ring
2,#4746,paid,2018-07-16 12:31:00,unfulfilled,yes,usd,100.5,13.64,0.0,114.14,nodiscount,0.0,fedex ground,2018-07-16 12:31:00,1,tri tone cubic zirconia paved simple band ring...,10.50,10.50,27281,pending,475 calle retama,475 calle retama,t-trendz,brownsville,78520,tx,us,nonotes,1,shopify payments,0.0,impression,low,web,size 8,ring,impression:ring
3,#4746,paid,2018-07-16 12:31:00,unfulfilled,yes,usd,100.5,13.64,0.0,114.14,nodiscount,0.0,fedex ground,2018-07-16 12:31:00,1,geometric tortoise link long necklace - brown,9.75,9.75,27976,pending,475 calle retama,475 calle retama,t-trendz,brownsville,78520,tx,us,nonotes,1,shopify payments,0.0,ella,low,web,brown,necklace,ella:necklace
4,#4746,paid,2018-07-16 12:31:00,unfulfilled,yes,usd,100.5,13.64,0.0,114.14,nodiscount,0.0,fedex ground,2018-07-16 12:31:00,1,acetate skinny bangle bracelet with front clos...,5.25,5.25,28943,pending,475 calle retama,475 calle retama,t-trendz,brownsville,78520,tx,us,nonotes,1,shopify payments,0.0,stylist,low,web,white/brown,bracelet,stylist:bracelet


In [145]:
x=usjh_all.groupby('vendor_category')['lineitem_name']
usjh_all_item={}

for key, value in x:
    usjh_all_item[key]=set(value)

In [152]:
len(usjh_all_item.keys())

326

### calculate frequency of each item

In [234]:
#create a list containing all unique items
all_items=usjh_all['lineitem_name']
all_items=set(all_items)
len(all_items)


26218

In [228]:
#calculate freqency
item_fre={'lineitem_name':[], 'frequency':[]}
for item in all_items:
    item_fre['lineitem_name'].append(item)
    item_fre['frequency'].append(len(usjh_all[usjh_all['lineitem_name']==item].name.unique())/len(usjh.name.unique()))

In [230]:
item_fre=pd.DataFrame(item_fre)
item_fre=item_fre.sort_values('frequency')

In [232]:
#merge item_fre with usjh_all
usjh_m=pd.merge(usjh_all, item_fre, on='lineitem_name', how='left')

In [233]:
#check missing values
usjh_m['frequency'].isnull().sum()

0

In [235]:
usjh_m.columns

Index(['name', 'financial_status', 'paid_at', 'fulfillment_status',
       'accepts_marketing', 'currency', 'subtotal', 'shipping', 'taxes',
       'total', 'discount_code', 'discount_amount', 'shipping_method',
       'created_at', 'lineitem_quantity', 'lineitem_name', 'lineitem_price',
       'lineitem_compare_at_price', 'lineitem_sku',
       'lineitem_fulfillment_status', 'shipping_street', 'shipping_address1',
       'shipping_company', 'shipping_city', 'shipping_zip',
       'shipping_province', 'shipping_country', 'notes', 'cancelled',
       'payment_method', 'refunded_amount', 'vendor', 'risk_level', 'source',
       'color', 'category', 'vendor_category', 'frequency'],
      dtype='object')

In [239]:
#create final association rule data frame for Flask
usjh_AR=usjh_m[['name','lineitem_name','lineitem_price','shipping_company','vendor','category', 'vendor_category', 'frequency']]
usjh_AR.shape

(83509, 8)

### how to perform recommandation
1. create an order list
2. for each item in the list, create an list containing its associated items (the highest support)
3. check if there is sale (discount) among these associated items
   a) if yes, recommand sale items
   b) if no, return the highest support item

In [517]:
#create sale list, randomly select 100 items as sale item
sale=usjh_AR[['lineitem_name','vendor_category','frequency']].sample(10).sort_values('frequency',ascending=False)
sale

,lineitem_name,vendor_category,frequency
22824,various charm bohemian bracelet,us jewelry house:bracelet,0.006599
31545,knit poncho with front pockets - bk,barring:apparel,0.003024
34083,druzy stone layered thread tassel earrings - b...,cloie:earring,0.002750
65758,cut out bohemian earrings - matte gold,it's sense:earring,0.002475
8670,rhinestone lariat y necklace - turquoise,us jewelry house:necklace,0.001100
82809,antique metal bar collar - patina,sugar:necklace,0.001100
73082,triple heart charm velvet choker,us jewelry house:necklace,0.001100
76930,multi pendant bohemian necklace - turquoise,us jewelry house:necklace,0.000825
30794,cz round pendant brass necklace - rose gold,lamant:necklace,0.000825
50521,pearl bouquet pin brooch,fashion jewelry:earring,0.000550


In [518]:
#create sale (vendor:category) basing on sale items
sale_vc=[]
for item in sale['lineitem_name']:
    sale_vc.append(usjh_AR[usjh_AR['lineitem_name']==item].vendor_category.iloc[0])
    
sale_vc=set(sale_vc)

In [519]:
sale_vc

{'barring:apparel',
 'cloie:earring',
 'fashion jewelry:earring',
 "it's sense:earring",
 'lamant:necklace',
 'sugar:necklace',
 'us jewelry house:bracelet',
 'us jewelry house:necklace'}

In [520]:
#create a dummy order list
order=usjh_AR[['lineitem_name']].sample(10)

In [521]:
#create order lhs basing on order items
order_vc=[]
for item in order['lineitem_name']:
    order_vc.append(usjh_AR[usjh_AR['lineitem_name']==item].vendor_category.iloc[0])
    
order_vc=set(order_vc)

In [522]:
order_vc

{'dorothy:earring',
 "it's sense:necklace",
 'm:necklace',
 'moment:earring',
 'pomina:earring',
 'ruff-skch:necklace',
 'us jewelry house:bracelet',
 'us jewelry house:earring',
 'us jewelry house:necklace'}

### conduct two recommendatins
- 1. basing on the highest support
- 2. basing on the highest confidence

#### recommendation basing on the highest support

In [523]:
#look up associated item rhs basing on the highst support
#use the rules_ which is sorted by support
assoc_rules={'rhs':[], 'support':[]}
for item in order_vc:
    if len(rules_[rules_['lhs']==item].rhs)>0: #some items have no rule
        
        assoc_rules['rhs'].append(rules_[rules_['lhs']==item].rhs.iloc[0])
        assoc_rules['support'].append(rules_[rules_['lhs']==item].support.iloc[0])

assoc_rules=pd.DataFrame(assoc_rules)

In [524]:
rhs=assoc_rules.sort_values('support', ascending=False)
rhs

,rhs,support
0,us jewelry house:earring,0.251306
2,us jewelry house:necklace,0.251306
5,us jewelry house:necklace,0.204014
4,us jewelry house:earring,0.118779
3,us jewelry house:earring,0.102007
6,cloie:earring,0.093484
1,us jewelry house:necklace,0.091559
7,us jewelry house:necklace,0.029970


In [525]:
#check if there is on sale item in rhs
recommend=''
r_support=0
for item in rhs['rhs']:
    #if there are items on sale in rhs, return the one with the greatest support
    if item in sale_vc:
        if rhs[rhs['rhs']==item].support.iloc[0] > r_support:
            recommend=item
            r_support=rhs[rhs['rhs']==item].support.iloc[0]

In [526]:
print(recommend)
print(r_support)

us jewelry house:necklace
0.251306021446247


In [527]:
#if recommend is not empty, which means this is associated on item on sale, recommand the top 3
if recommend != '':
    rec_items=sale[sale['vendor_category']==recommend].sort_values('frequency',ascending=False)
    #if len(rec_items['lineitem_name'].unique())>3:
        #rec_items=rec_items['lineitem_name'].unique()[:3]
    #else:
    rec_items=rec_items['lineitem_name'].unique()
#if there is no on sale item, the recommend will be an empty string, 
#then just simply choose the item with the highest support in rhs
else :
    recommend=rhs['rhs'].iloc[0]
#now we have the final recommend, pick the specific items to make recommendation
#find the lineitem name in recommend with top 3 frequency
    
    rec_items=usjh_AR[usjh_AR['vendor_category']==recommend].sort_values('frequency',ascending=False).lineitem_name
#there are a lot of duplicated items, get the unique top 3
    rec_items=rec_items.unique()[:3]

    

In [528]:
rec_items=rec_items.tolist()

In [529]:
#we want to alway recommend 3 items
x=usjh_AR[usjh_AR['vendor_category']==rhs.rhs.iloc[0]].sort_values('frequency',ascending=False).lineitem_name
x=x.unique()

dif=3-len(rec_items)
dif
if dif==1:
    rec_items.append(x[0])
elif dif==2:
    rec_items.append(x[0])
    rec_items.append(x[1])
else:
    rec_items=rec_items[:3]

In [530]:
rec_items

['rhinestone lariat y necklace - turquoise',
 'triple heart charm velvet choker',
 'multi pendant bohemian necklace - turquoise']

#### recommendation basing on the highest confidence

In [531]:
#look up associated item rhs basing on the highst confidence
#use the rules_ which is sorted by confidence
assoc_rulec={'rhs':[], 'confidence':[]}
for item in order_vc:
    if len(rulec_[rulec_['lhs']==item].rhs)>0: #some items have no rule
        
        assoc_rulec['rhs'].append(rulec_[rulec_['lhs']==item].rhs.iloc[0])
        assoc_rulec['confidence'].append(rulec_[rulec_['lhs']==item].confidence.iloc[0])

assoc_rulec=pd.DataFrame(assoc_rulec)

In [532]:
rhs=assoc_rulec.sort_values('confidence', ascending=False)
rhs

,rhs,confidence
4,us jewelry house:earring,0.759227
5,us jewelry house:necklace,0.740519
6,cloie:earring,0.715789
1,us jewelry house:necklace,0.711538
2,us jewelry house:necklace,0.641854
3,us jewelry house:earring,0.629881
7,us jewelry house:necklace,0.589189
0,us jewelry house:earring,0.495125


In [533]:
#check if there is on sale item in rhs
recommend_=''
r_confidence=0
for item in rhs['rhs']:
    #if there are items on sale in rhs, return the one with the greatest confidence
    if item in sale_vc:
        if rhs[rhs['rhs']==item].confidence.iloc[0] > r_confidence:
            recommend_=item
            r_confidence=rhs[rhs['rhs']==item].confidence.iloc[0]

In [534]:
print(recommend_)
print(r_confidence)

us jewelry house:necklace
0.740518962075848


In [535]:
#if recommend_ is not empty, which means this is associated on item on sale, recommand the top 3 associated items in the sale list
if recommend_ != '':
    rec_items_=sale[sale['vendor_category']==recommend_].sort_values('frequency',ascending=False)
    if len(rec_items_['lineitem_name'].unique())>3:
        rec_items_=rec_items_['lineitem_name'].unique()[:3]
    else:
        rec_items_=rec_items_['lineitem_name'].unique()
#if there is no on sale item, the recommend_ will be an empty string, 
#then just simply choose the item with the highest confidence in rhs
else :
    recommend_=rhs['rhs'].iloc[0]
#now we have the final recommend, pick the specific items to make recommendation
#find the lineitem name in recommend with top 3 frequency
    rec_items_=usjh_AR[usjh_AR['vendor_category']==recommend_].sort_values('frequency',ascending=False).lineitem_name
#there are a lot of duplicated items, get the unique top 3
    rec_items_=rec_items_.unique()[:3]

In [536]:
rec_items_=rec_items_.tolist()

In [537]:
#we want to alway recommend 3 items
x=usjh_AR[usjh_AR['vendor_category']==rhs.rhs.iloc[0]].sort_values('frequency',ascending=False).lineitem_name
x=x.unique()

dif=3-len(rec_items_)
dif
if dif==1:
    rec_items_.append(x[0])
elif dif==2:
    rec_items_.append(x[0])
    rec_items_.append(x[1])
else:
    rec_items_=rec_items_[:3]
    

In [538]:
rec_items_

['rhinestone lariat y necklace - turquoise',
 'triple heart charm velvet choker',
 'multi pendant bohemian necklace - turquoise']

In [539]:
recommendation=set(rec_items+rec_items_)

In [540]:
recommendation

{'multi pendant bohemian necklace - turquoise',
 'rhinestone lariat y necklace - turquoise',
 'triple heart charm velvet choker'}